In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from Segment import Body, Segment

In [2]:
# build our body 

tree = Segment(171380, 4.305, 59.539, "Torso")
tree.add_child(51.70, Segment(17556.0, 13.423, 17.3, "Head"))
tree.add_child(5.45, 
               Segment(9041.34, 0.814, 1.264, "LUArm").add_child(2.65,
               Segment(5739.14, 0.534, 0.271, "LFArm").add_child(1.20,
               Segment(1400.3, 0.369, 0.140, "LHand"))))
tree.add_child(5.45, 
               Segment(9041.34, 0.814, 1.264, "RUArm").add_child(2.65,
               Segment(5739.14, 0.534, 0.271, "RFArm").add_child(1.20,
               Segment(1400.3, 0.369, 0.140, "RHand"))))
tree.add_child(2.72, 
               Segment(29314.3, 1.640, 1.440, "LThigh").add_child(0.85,
               Segment(13973.7, 0.879, 0.380, "LLeg").add_child(0.56,
               Segment(2006.4, 0.456, 0.313, "LFoot"))))
tree.add_child(2.72, 
               Segment(29314.3, 1.640, 1.440, "RThigh").add_child(0.85,
               Segment(13973.7, 0.879, 0.380, "RLeg").add_child(0.56,
               Segment(2006.4, 0.456, 0.313, "RFoot"))))
body = Body(tree)

In [3]:
# Now build our functions for the warm parameter and cold parameter

def wrm(btree, bpram):
    """Defines the warm parameter, a weighted sum of temperature
    errors over the body
    """
    temps = btree.get_param("temp", True)
    tsets = btree.get_param("tset", True)
    skwts = btree.get_param("skwt", True)
    # induvidual wars are allways positive or zero, temp > tset
    err = np.clip(temps - tsets, 0, None)
    wmt = skwts * err
    return np.sum(wmt)

def cld(btree, bpram):
    """Defines the cold parameter, a weighted sum of temperature
    erros over the body
    """
    temps = btree.get_param("temp", True)
    tsets = btree.get_param("tset", True)
    skwts = btree.get_param("skwt", True)
    # induvidual wars are allways negative or zero, temp < tset
    err = np.clip(temps - tsets, None, 0)
    cldt = skwts * -err
    return np.sum(cldt)

# add the parameters to the body
tsets = [35.0, 36.1, 34.4, 34.9, 35.3, 34.4, 34.9, 35.3,
                     34.8, 34.3, 34.6, 34.8, 34.3, 34.6]
skwts = [0.493, 0.070, 0.023, 0.012, 0.092, 0.023, 0.012, 0.092,
                       0.050, 0.025, 0.017, 0.050, 0.025, 0.017]
chilf = [0.020, 0.850, 0.004, 0.026, 0.000, 0.004, 0.026, 0.000,
                       0.023, 0.012, 0.000, 0.023, 0.012, 0.000]

body.body_tree.set_param("tset", tsets)
body.body_tree.set_param("skwt", skwts)
body.body_tree.set_param("chilf", chilf)

# and register the warm and cld parameters
body.register_body_parameter("wrm", wrm, 0.0)
body.register_body_parameter("cld", cld, 0.0)

In [ ]:
# add in new component values and then a new matrix function for
# all matrixes

ebases = 